In [11]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from urllib.parse import urljoin

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import time

In [12]:
def get_page_content(url, max_retries=3, delay=5):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'
    }
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            return response.text
        except requests.exceptions.RequestException as e:
            print(f"Error fetching the page: {e}")
            retries += 1
            time.sleep(delay)
    return None

def extract_article_content(soup):
    # Remove unwanted elements like scripts, styles, and comments
    for element in soup(["script", "style", "meta", "noscript", "iframe", "aside", "header", "footer", "nav"]):
        element.decompose()

    # Find the main content container
    main_content = None
    content_tags = ["article", "div", "section", "main"]
    for tag in content_tags:
        main_content = soup.find(tag, {"class": re.compile("(article|post|content|entry|body|story)", re.IGNORECASE)})
        if main_content:
            break

    if main_content is None:
        main_content = soup.find("body")

    # Extract paragraphs from the main content
    paragraphs = main_content.find_all("p")
    article_content = "\n".join([p.get_text(strip=True) for p in paragraphs])

    # If the extracted content is too short, try extracting from the whole page
    if len(article_content) < 200:
        paragraphs = soup.find_all("p")
        article_content = "\n".join([p.get_text(strip=True) for p in paragraphs])

    # Remove empty lines and extra whitespace
    article_content = re.sub(r"\n+", "\n", article_content).strip()
    article_content = re.sub(r"\s+", " ", article_content)

    return article_content

def extract_article_title(soup):
    title_element = soup.find('h1')
    if title_element:
        return title_element.get_text(strip=True)
    return ""

def extract_article_date(soup):
    date_element = soup.find('time')
    if date_element:
        datetime_str = date_element.get('datetime')
        if datetime_str:
            try:
                return datetime.fromisoformat(datetime_str).strftime("%Y-%m-%d")
            except ValueError:
                pass
    return ""

def extract_article_author(soup):
    author_element = soup.find(class_=re.compile('author', re.IGNORECASE))
    if author_element:
        return author_element.get_text(strip=True)
    return ""

def extract_article_images(soup, base_url):
    img_tags = soup.find_all('img')
    image_urls = []
    for img in img_tags:
        img_url = img.get('src')
        if img_url:
            absolute_url = urljoin(base_url, img_url)
            image_urls.append(absolute_url)
    return image_urls



In [13]:
from gnews import GNews

google_news = GNews(language='es', 
                    country='Mexico', 
                    period='7d', 
                    start_date=None, 
                    end_date=None, 
                    max_results=5, 
                    #exclude_websites=['yahoo.com', 'cnn.com'],
                    #proxy=proxy
                    )
news = google_news.get_news('espanol Claudia Sheinbaum')

In [14]:
def extract_article_content_selenium(url,proxy=None):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run Chrome in headless mode
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36')
    if proxy is not None:
        options.add_argument(f'--proxy-server={proxy}')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    # Find the article content using common HTML tags and classes
    article_tags = [
        {'name': 'article'},
        {'name': 'div', 'class': 'article-body'},
        {'name': 'div', 'class': 'article-content'},
        {'name': 'div', 'class': 'entry-content'},
        {'name': 'div', 'class': 'post-content'},
        {'name': 'div', 'class': 'story-body'},
        {'name': 'div', 'itemprop': 'articleBody'},
        {'name': 'div', 'id': 'article-body'},
        {'name': 'div', 'class': 'article-text'},
        {'name': 'div', 'class': 'post-text'},
        {'name': 'div', 'class': 'post-body'},
        {'name': 'div', 'class': 'rich-text'},
        {'name': 'div', 'class': 'article-content'},
        {'name': 'section', 'class': 'article-body'},
        {'name': 'section', 'class': 'post-content'},
        {'name': 'section', 'class': 'entry-content'},
    ]

    article_content = ''
    for tag in article_tags:
        try:
            if 'class' in tag:
                article_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, tag['class']))
                )
            elif 'id' in tag:
                article_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, tag['id']))
                )
            elif 'itemprop' in tag:
                article_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//*[@itemprop='{tag['itemprop']}']"))
                )
            else:
                article_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, tag['name']))
                )
            
            paragraphs = article_element.find_elements(By.TAG_NAME, 'p')
            article_content = '\n'.join([p.text for p in paragraphs])
            
            if article_content:
                break
        except (TimeoutException, StaleElementReferenceException):
            continue

    if not article_content:
        # If the article content is still not found, try to extract paragraphs from the entire page
        try:
            paragraphs = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, 'p'))
            )
            article_content = '\n'.join([p.text for p in paragraphs])
        except (TimeoutException, StaleElementReferenceException):
            pass

    driver.quit()

    # Remove empty lines and extra whitespace
    article_content = re.sub(r"\n+", "\n", article_content).strip()
    article_content = re.sub(r"\s+", " ", article_content)

    return article_content

def extract_article_content_selenium_js(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run Chrome in headless mode
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    # Wait for the page to load and JavaScript to render the content
    time.sleep(5)  # Adjust the delay as needed

    # Find the article content using common HTML tags and classes
    article_tags = [
        {'name': 'article'},
        {'name': 'div', 'class': 'article-body'},
        {'name': 'div', 'class': 'article-content'},
        {'name': 'div', 'class': 'entry-content'},
        {'name': 'div', 'class': 'post-content'},
        {'name': 'div', 'class': 'story-body'},
        {'name': 'div', 'itemprop': 'articleBody'},
        {'name': 'div', 'id': 'article-body'},
        {'name': 'div', 'class': 'article-text'},
        {'name': 'div', 'class': 'post-text'},
        {'name': 'div', 'class': 'post-body'},
        {'name': 'div', 'class': 'rich-text'},
        {'name': 'div', 'class': 'article-content'},
        {'name': 'section', 'class': 'article-body'},
        {'name': 'section', 'class': 'post-content'},
        {'name': 'section', 'class': 'entry-content'},
    ]

    article_content = ''
    for tag in article_tags:
        try:
            if 'class' in tag:
                article_element = driver.find_element(By.CLASS_NAME, tag['class'])
            elif 'id' in tag:
                article_element = driver.find_element(By.ID, tag['id'])
            elif 'itemprop' in tag:
                article_element = driver.find_element(By.XPATH, f"//*[@itemprop='{tag['itemprop']}']")
            else:
                article_element = driver.find_element(By.TAG_NAME, tag['name'])
            
            paragraphs = article_element.find_elements(By.TAG_NAME, 'p')
            article_content = '\n'.join([p.text for p in paragraphs])
            
            if article_content:
                break
        except (NoSuchElementException, StaleElementReferenceException):
            continue

    if not article_content:
        # If the article content is still not found, try to extract paragraphs from the entire page
        try:
            paragraphs = driver.find_elements(By.TAG_NAME, 'p')
            article_content = '\n'.join([p.text for p in paragraphs])
        except (NoSuchElementException, StaleElementReferenceException):
            pass

    driver.quit()

    # Remove empty lines and extra whitespace
    article_content = re.sub(r"\n+", "\n", article_content).strip()
    article_content = re.sub(r"\s+", " ", article_content)

    return article_content

In [15]:
def scrape_news_article(url):
    page_content = get_page_content(url)
    if page_content:
        soup = BeautifulSoup(page_content, 'html.parser')

        article_title = extract_article_title(soup)
        article_date = extract_article_date(soup)
        article_author = extract_article_author(soup)
        
        # Try extracting content using the existing function first
        article_content = extract_article_content_selenium(url)
        
        # If the content is empty, try the new function for JavaScript-rendered content
        if not article_content.strip():
            article_content = extract_article_content_selenium_js(url)
        
        article_images = extract_article_images(soup, url)

        article_data = {
            'title': article_title,
            'date': article_date,
            'author': article_author,
            'content': article_content,
            'images': article_images
        }
        return article_data
    return None

In [5]:
# Example usage
url = news[1]['url']
print(url)
article_data = scrape_news_article(url)
if article_data:
    print("Title:", article_data['title'])
    print("Date:", article_data['date'])
    print("Author:", article_data['author'])
    print("Content:", article_data['content'])
    print("Images:", article_data['images'])
else:
    print("Failed to scrape the article.")

https://news.google.com/rss/articles/CBMijwFodHRwczovL3d3dy5sb2NhbDEwLmNvbS9lc3Bhbm9sLzIwMjQvMDMvMDEvYXJyYW5jYS1jYW1wYW5hLXBvci1sYS1wcmVzaWRlbmNpYS1kZS1tZXhpY28tZW50cmUtb2ZlcnRhcy1kZS1jb250aW51aWRhZC15LWNyaXRpY2FzLWEtbGEtdmlvbGVuY2lhL9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en


03/07/2024 04:09:33 PM - ====== WebDriver manager ======
/bin/sh: line 1: google-chrome: command not found
/bin/sh: line 1: google-chrome: command not found
03/07/2024 04:09:33 PM - Get LATEST chromedriver version for google-chrome
03/07/2024 04:09:33 PM - Get LATEST chromedriver version for google-chrome
03/07/2024 04:09:33 PM - Driver [/home/uumami/.wdm/drivers/chromedriver/linux64/122.0.6261.111/chromedriver-linux64/chromedriver] found in cache


Title: 
Date: 
Author: 
Content: MEXICO CITY – México arrancó el viernes oficialmente las campañas para las elecciones con más cargos en juego en la historia del país que podrían, por primera vez, llevar a una mujer a la presidencia. A la candidata oficialista Claudia Sheinbaum, que representa la promesa de continuidad del proyecto político del presidente Andrés Manuel López Obrador, la esperaban decenas de miles de personas en una abarrotada plaza del Zócalo, en la Ciudad de México. Por su parte, la opositora Xóchitl Gálvez, que defiende un “México sin miedo”, llevó su inicio de campaña a los estados de Guanajuato y Zacatecas, azotados por la violencia. Recommended Videos Dentro de tres meses, los mexicanos acudirán a las urnas para elegir a quien gobernará el país por los siguientes seis años, pero los comicios del 2 de junio también definirán los liderazgos en ocho gobernaciones, en la alcaldía de la capital, en 20.000 cargos federales y locales, y también serán elegidos 500 diputad

In [6]:
print(article_data['content'])

MEXICO CITY – México arrancó el viernes oficialmente las campañas para las elecciones con más cargos en juego en la historia del país que podrían, por primera vez, llevar a una mujer a la presidencia. A la candidata oficialista Claudia Sheinbaum, que representa la promesa de continuidad del proyecto político del presidente Andrés Manuel López Obrador, la esperaban decenas de miles de personas en una abarrotada plaza del Zócalo, en la Ciudad de México. Por su parte, la opositora Xóchitl Gálvez, que defiende un “México sin miedo”, llevó su inicio de campaña a los estados de Guanajuato y Zacatecas, azotados por la violencia. Recommended Videos Dentro de tres meses, los mexicanos acudirán a las urnas para elegir a quien gobernará el país por los siguientes seis años, pero los comicios del 2 de junio también definirán los liderazgos en ocho gobernaciones, en la alcaldía de la capital, en 20.000 cargos federales y locales, y también serán elegidos 500 diputados y 128 senadores. Las dos favor

In [16]:
# Example usage
url = news[0]['url']
print(url)
article_data = scrape_news_article(url)
if article_data:
    print("Title:", article_data['title'])
    print("Date:", article_data['date'])
    print("Author:", article_data['author'])
    print("Content:", article_data['content'])
    print("Images:", article_data['images'])
else:
    print("Failed to scrape the article.")

https://news.google.com/rss/articles/CBMiVmh0dHBzOi8vd3d3Lm55dGltZXMuY29tL2VzLzIwMjQvMDMvMDEvZXNwYW5vbC9lbGVjY2lvbmVzLW1leGljby1jbGF1ZGlhLXNoZWluYmF1bS5odG1s0gEA?oc=5&hl=en-US&gl=US&ceid=US:en


03/07/2024 04:17:59 PM - ====== WebDriver manager ======
/bin/sh: line 1: google-chrome: command not found
/bin/sh: line 1: google-chrome: command not found
03/07/2024 04:17:59 PM - Get LATEST chromedriver version for google-chrome
03/07/2024 04:18:00 PM - Get LATEST chromedriver version for google-chrome
03/07/2024 04:18:00 PM - Driver [/home/uumami/.wdm/drivers/chromedriver/linux64/122.0.6261.111/chromedriver-linux64/chromedriver] found in cache
03/07/2024 04:20:55 PM - ====== WebDriver manager ======
/bin/sh: line 1: google-chrome: command not found
/bin/sh: line 1: google-chrome: command not found
03/07/2024 04:20:55 PM - Get LATEST chromedriver version for google-chrome
03/07/2024 04:20:55 PM - Get LATEST chromedriver version for google-chrome
03/07/2024 04:20:56 PM - Driver [/home/uumami/.wdm/drivers/chromedriver/linux64/122.0.6261.111/chromedriver-linux64/chromedriver] found in cache


Title: 
Date: 
Author: 
Content: Claudia Sheinbaum, la protegida del presidente de México, mantiene una amplia ventaja en las encuestas. Pero para muchos en México, sigue siendo un misterio. Por Simon Romero y Emiliano Rodríguez Mega Reportando desde Ciudad de México Con las elecciones presidenciales de México a realizarse en apenas tres meses, hay algo claro: la candidata del partido gobernante parece ser la clara ganadora. Claudia Sheinbaum, física y protegida política del presidente actual, mantiene una amplia ventaja de cerca de 30 puntos porcentuales en las encuestas sobre la candidata de la oposición, Xóchitl Gálvez, empresaria del sector tecnológico. Este viernes es el inicio oficial de la campaña. Jugando a lo seguro en un momento en el que el presidente saliente, Andrés Manuel López Obrador, sigue teniendo altos niveles de popularidad, Sheinbaum se ha mantenido tan cerca de sus políticas y su personalidad que no solo se ha comprometido a adoptar las prioridades del presidente,

In [17]:
print(article_data['content'])

Claudia Sheinbaum, la protegida del presidente de México, mantiene una amplia ventaja en las encuestas. Pero para muchos en México, sigue siendo un misterio. Por Simon Romero y Emiliano Rodríguez Mega Reportando desde Ciudad de México Con las elecciones presidenciales de México a realizarse en apenas tres meses, hay algo claro: la candidata del partido gobernante parece ser la clara ganadora. Claudia Sheinbaum, física y protegida política del presidente actual, mantiene una amplia ventaja de cerca de 30 puntos porcentuales en las encuestas sobre la candidata de la oposición, Xóchitl Gálvez, empresaria del sector tecnológico. Este viernes es el inicio oficial de la campaña. Jugando a lo seguro en un momento en el que el presidente saliente, Andrés Manuel López Obrador, sigue teniendo altos niveles de popularidad, Sheinbaum se ha mantenido tan cerca de sus políticas y su personalidad que no solo se ha comprometido a adoptar las prioridades del presidente, sino que en ocasiones imita su p